In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image
import os
import pathlib
import numpy as np
import pathlib
import shutil

In [0]:
BASE_PATH = ''
DATA_PATH = ''

batch_size = 128
img_height = 240
img_width = 320

resized_height = 120
resized_width = 160

num_classes = 2
NUM_EPOCHS = 120
seed = 12
log_dir = BASE_PATH + "/logs"

STEPS_PER_EPOCH = 1508 //batch_size

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.2),
  layers.experimental.preprocessing.RandomContrast(0.2),
])

resize = tf.keras.layers.experimental.preprocessing.Resizing(resized_height, resized_width)

def preprocess_input(image):
    return (image/127.5) - 1

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=seed,
  shuffle=True,
  validation_split=0.1,
  subset='training',
  image_size=(img_height, img_width)
)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

train_ds = train_ds.map(
  lambda x, y: (preprocess_input(x), y))


val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=seed,
  shuffle=True,
  validation_split=0.1,
  subset='validation',
  image_size=(img_height, img_width)
)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(
  lambda x, y: (preprocess_input(x), y))

Found 16468 files belonging to 2 classes.
Using 14822 files for training.
Found 16468 files belonging to 2 classes.
Using 1646 files for validation.

In [0]:
class CustomLearningRateScheduler(tf.keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # (epoch to start, learning rate) tuples
    (10, 0.005),
    (30, 0.001),
    (50, 0.0005)
]


def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

In [0]:
def get_model(lr=0.001, alpha=0.35):
    base_model = tf.keras.applications.ResNet152V2(
        input_shape=(img_height, img_width, 3),
        include_top=False, weights='imagenet', 
#       alpha=alpha
    )
#     base_model.trainable = False
    base_model.trainable = True
    # Add a new classifier layer for transfer learning
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    dropout = tf.keras.layers.Dropout(0.2)
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
#     prediction_layer = tf.keras.layers.Dense(1, activation='softmax')
  
    model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      dropout,
      prediction_layer
    ])
    return model
  
def get_compiled_model(alpha=0.35, lr=0.001):
    model = get_model(lr, alpha)
#     lr = learning_rate = CustomSchedule(2)
    model.compile(
                optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9),
#                 optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model
model = get_model()
model.summary()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3534976955442343> in <module> 
 28 metrics=['accuracy'])
 29 return model
 ---> 30 model = get_model ( ) 
 31 model . summary ( ) 

 <command-3534976955442343> in get_model (lr, alpha) 
 2 base_model = tf.keras.applications.ResNet152V2(
 3 input_shape = ( img_height , img_width , 3 ) , 
 ----> 4 include_top=False, weights='imagenet', alpha=alpha)
 5 # base_model.trainable = False 
 6 base_model . trainable = True 

 TypeError : ResNet152V2() got an unexpected keyword argument 'alpha'

In [0]:
def train_and_evaluate(train_ds, val_ds=None, lr=0.001):
    model = get_compiled_model()
#     model = get_compiled_model_2()
    checkpoint_path = BASE_PATH + "/Model/Best_Model.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
#     model.save_weights(checkpoint_path.format(epoch=0))

    # 创建一个保存模型权重的回调
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only= True,
                                                     save_best_only = True,
                                                     monitor='accuracy',
                                                     mode='auto', 
                                                     save_freq='epoch',
                                                     verbose=1)
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)
    board_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,  # How often to log histogram visualizations
        embeddings_freq=0,  # How often to log embedding visualizations
        update_freq="epoch",
    )  # How often to write logs (default: once per epoch)
    
#     steps_per_epoch = len(train_ds) // batch_size
    hist = model.fit(train_ds, 
#                      steps_per_epoch=steps_per_epoch,
                     epochs=NUM_EPOCHS,
                     validation_data=val_ds,
#                      class_weight={0:0.3, 1:0.7},
#                      validation_steps=validation_steps,
                     verbose=2,
                     callbacks=[
                       cp_callback, 
#                        board_callback, 
                       early_stop,
#                        CustomLearningRateScheduler(lr_schedule),
                     ])
#     model.save('saved_model/my_model')
    return hist,model

In [0]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
      print('yes')
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [0]:
tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  hist,model = train_and_evaluate(train_ds)

In [0]:
import matplotlib.pyplot as plt
acc = hist.history['accuracy']
# val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
# val_loss = hist.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()